In [1]:
import random
from Multi_Random_Forest1 import *
import time
import pandas as pd
from Portal import *

Este módulo usa pandas.Dataframe
E está feito para apenas prever variáveis categóricas 0/1


In [2]:
pd.set_option('display.max_columns',1000)
pd.set_option('display.max_rows',1000)


In [3]:
def remove_columns(file):
    file=file.drop(['League','Date','Time','HT','AT'],axis=1)
    try:
        file=file.drop(['ODDH_Aver.','ODDD_Aver.','ODDA_Aver.'],axis=1)
    except:
        pass
    return file

def train_test_split(file,test_percentage=10,end=False):
    if end:
        size=len(file.index)
        print(size,'size')
        test_size=int((size*test_percentage)/100)
        print(test_size,'test_size')
        Test=file[(-test_size):].reset_index(drop=True)
        Train=file[:(-test_size)].reset_index(drop=True)
        print('len(Test): ',len(Test),'Total len: ',len(Test)+len(Train))
        return Train, Test    
    else:   
        size=len(file.index)
        print(size,'size')
        test_size=int((size*test_percentage)/100)
        print(test_size,'test_size')
        limit=size-test_size
        print(limit)
        test_ind=random.randrange(0,limit)
        print(test_ind,test_ind+test_size)
        Test=file[test_ind:(test_ind+test_size)].reset_index(drop=True)
        Train=pd.concat([file[:test_ind],file[(test_ind+test_size):]],sort=False).reset_index(drop=True)
        return Train, Test
    
def Covid_Test(df_Test):
# para eliminar os jogos do intervalo do Covid
    df = df_Test.sort_values(['Date','Time'],ascending=[True,True])
    maxi=[0,0]
    for i in range(len(df)-1):
        ok = abs((df.iloc[i]['Date']-df.iloc[i+1]['Date']).days)
        print(ok)
        if (ok>maxi[0]) and not(6<(df.iloc[i+1]['Date'].month)<9) and not(11<(df.iloc[i+1]['Date'].month)<13):
            maxi[0]=ok
            maxi[1]=i+1
    teams = list(np.unique(df['HT']))
    print(maxi[0])
    print(df.iloc[maxi[1]-1],df.iloc[maxi[1]])
    if maxi[0]<30:
        return df
    for t in teams:
        vars()[t]=0
    dropz=[]
    for i in range(maxi[1],len(df)):
        HT = df.iloc[i]['HT']
        AT = df.iloc[i]['AT']
        if (vars()[AT]<5) or (vars()[HT]<5):
            dropz+=[i]
        vars()[AT]+=1
        vars()[HT]+=1
    df = df.drop(dropz,axis=0)
    return df.sort_values(['Date','Time'],ascending=[True,True]).reset_index(drop=True)

In [31]:
ll=pd.read_excel('Sky_Championship.xlsx',0)

In [32]:
bund_Train,bund_Test=train_test_split(ll,test_percentage=10,end=True)

3836 size
383 test_size
len(Test):  383 Total len:  3836


In [49]:
bund_Test.tail(5)

,League,Date,Time,HT,AT,Colx,HT_HPoints_5,AT_APoints_5,HA_HAPointsDiff,HT_TotalPoints_5,AT_TotalPoints_5,HA_TotalPointsDiff,SQuality_H5,SQuality_A5,SQuality_Diff5,HS_5,Last_H_S,HST_5,Last_H_ST,HScrd_5,Last_H_Scrd,HConcd_5,Last_H_Concd,AS_5,Last_A_S,AST_5,Last_A_ST,AScrd_5,Last_A_Scrd,AConcd_5,Last_A_Concd,S_Diff_H-A_5,ST_Diff_H-A_5,Scrd_Diff_H-A_5,Concd_Diff_H-A_5,WinCounterTotal_H,WinCounterTotal_A,WinCounter_H,WinCounter_A,ScrdCounterTotal_H,ScrdCounterTotal_A,ScrdCounter_H,ScrdCounter_A,ConcdCounterTotal_H,ConcdCounterTotal_A,ConcdCounter_H,ConcdCounter_A,Max_WinCounterTotal_H,Max_WinCounterTotal_A,Max_WinCounter_H,Max_WinCounter_A,Max_ScrdCounterTotal_H,Max_ScrdCounterTotal_A,Max_ScrdCounter_H,Max_ScrdCounter_A,Max_ConcdCounterTotal_H,Max_ConcdCounterTotal_A,Max_ConcdCounter_H,Max_ConcdCounter_A,Min_WinCounterTotal_H,Min_WinCounterTotal_A,Min_WinCounter_H,Min_WinCounter_A,Min_ScrdCounterTotal_H,Min_ScrdCounterTotal_A,Min_ScrdCounter_H,Min_ScrdCounter_A,Min_ConcdCounterTotal_H,Min_ConcdCounterTotal_A,Min_ConcdCounter_H,Min_ConcdCounter_A,STForm_H5,STForm_A5,STForm_Diff5,CCCForm_H5,CCCForm_A5,CCCForm_Diff5,PassForm_H5,PassForm_A5,PassForm_Diff5
140,Dutch Eredivisie,2020-03-07,1945,Heerenveen,Ajax,1,0.7,0.8,-0.1,0.7,0.8,-0.1,2.4,1.5,0.9,23,0,9,1,2,1,2,0,22,0,5,0,1,0,1,1,1,4,1,1,1,-2,-5,-3,21,-2,11,-1,-16,-2,-8,-6,2,11,7,7,0,16,0,8,18,5,9,7,-9,0,0,0,-23,0,-12,0,0,-5,0,0,5,2,3,0,0,0,-1,5,-6
141,Dutch Eredivisie,2020-03-08,1115,FC Utrecht,Sparta Rotterdam,1,1.6,0.9,0.7,1.6,0.9,0.7,4.5,4.5,0.0,14,0,4,0,2,0,1,1,14,0,5,1,3,1,1,0,0,-1,-1,0,-1,2,2,1,9,4,3,1,-4,-1,-1,1,4,0,2,2,3,3,3,3,5,3,4,0,-2,-6,-5,-10,-12,-7,-5,-3,-3,-5,-2,-7,0,1,-1,-1,1,-2,-2,-9,7
142,Dutch Eredivisie,2020-03-08,1330,Feyenoord,Willem II,1,3.0,1.4,1.6,3.0,1.4,1.6,3.4,4.1,-0.7,17,0,7,0,2,0,1,0,11,0,4,0,2,1,2,0,6,3,0,-1,-1,1,6,-2,8,3,8,1,-3,-4,-1,-4,8,2,0,0,0,2,0,3,5,6,2,5,-2,-4,-8,0,-9,-4,-9,-2,-4,-1,-6,0,3,0,3,1,0,1,4,-1,5
143,Dutch Eredivisie,2020-03-08,1330,Heracles Almelo,RKC Waalwijk,1,1.9,0.3,1.6,1.9,0.3,1.6,0.8,1.2,-0.4,12,0,5,0,1,0,0,0,11,1,3,1,1,1,2,0,1,2,0,-2,-1,1,2,-4,-1,1,10,-3,2,-3,2,-4,4,0,3,3,5,8,0,7,0,4,0,5,-5,-11,-5,-2,0,-5,-12,0,-10,-7,-6,-2,-1,-1,0,-1,-1,0,-1,0,-1
144,Dutch Eredivisie,2020-03-08,1545,Groningen,PSV Eindhoven,1,1.8,1.8,0.0,1.8,1.8,0.0,2.8,4.5,-1.7,13,1,3,0,1,0,1,1,15,0,5,0,2,0,1,0,-2,-2,-1,0,-2,-1,-1,2,1,4,-1,2,-3,-2,-1,-1,4,5,5,5,5,5,8,3,5,4,5,2,-2,-3,-1,-9,-3,-6,0,-3,-2,-5,-1,-6,-2,1,-3,-1,1,-2,0,6,-6


In [9]:
bund_Test.tail(11)

,League,Date,Time,HT,AT,Colx,HT_HPoints_5,AT_APoints_5,HA_HAPointsDiff,HT_TotalPoints_5,AT_TotalPoints_5,HA_TotalPointsDiff,SQuality_H5,SQuality_A5,SQuality_Diff5,HS_5,Last_H_S,HST_5,Last_H_ST,HScrd_5,Last_H_Scrd,HConcd_5,Last_H_Concd,AS_5,Last_A_S,AST_5,Last_A_ST,AScrd_5,Last_A_Scrd,AConcd_5,Last_A_Concd,S_Diff_H-A_5,ST_Diff_H-A_5,Scrd_Diff_H-A_5,Concd_Diff_H-A_5,WinCounterTotal_H,WinCounterTotal_A,WinCounter_H,WinCounter_A,ScrdCounterTotal_H,ScrdCounterTotal_A,ScrdCounter_H,ScrdCounter_A,ConcdCounterTotal_H,ConcdCounterTotal_A,ConcdCounter_H,ConcdCounter_A,Max_WinCounterTotal_H,Max_WinCounterTotal_A,Max_WinCounter_H,Max_WinCounter_A,Max_ScrdCounterTotal_H,Max_ScrdCounterTotal_A,Max_ScrdCounter_H,Max_ScrdCounter_A,Max_ConcdCounterTotal_H,Max_ConcdCounterTotal_A,Max_ConcdCounter_H,Max_ConcdCounter_A,Min_WinCounterTotal_H,Min_WinCounterTotal_A,Min_WinCounter_H,Min_WinCounter_A,Min_ScrdCounterTotal_H,Min_ScrdCounterTotal_A,Min_ScrdCounter_H,Min_ScrdCounter_A,Min_ConcdCounterTotal_H,Min_ConcdCounterTotal_A,Min_ConcdCounter_H,Min_ConcdCounter_A,STForm_H5,STForm_A5,STForm_Diff5,CCCForm_H5,CCCForm_A5,CCCForm_Diff5,PassForm_H5,PassForm_A5,PassForm_Diff5
372,Sky Bet Championship,2020-07-22 00:00:00,1930.0,Wigan Athletic,Fulham,1,2.8,1.8,1.0,2.8,1.8,1.0,3.5,3.4,0.1,12,1,5,1,3,0,1,1,11,1,5,1,2,1,1,1,1,0,1,0,-1,1,4,-1,2,1,4,-1,-1,-1,6,2,4,3,0,0,7,9,0,5,8,5,0,1,-12,-5,-11,-3,-5,-3,-6,-1,-9,-5,-16,-7,1,2,-1,1,-1,2,1,6,-5
373,Sky Bet Championship,"3:00pm, Saturday 3rd August 2019.",NaN,Barnsley,Fulham,1,1.9,1.6,0.3,1.9,1.6,0.3,2.5,3.2,-0.7,15,0,4,0,1,1,1,0,13,1,5,0,2,0,1,0,2,-1,-1,0,2,-1,1,-2,2,2,1,1,-1,-2,4,-1,1,5,0,0,7,8,9,3,4,6,0,4,-19,-3,-9,-2,-5,-4,-3,-3,-17,-4,-20,-4,2,2,0,1,-1,2,-4,8,-12
374,Sky Bet Championship,"3:00pm, Saturday 3rd August 2019.",NaN,Blackburn Rovers,Charlton Athletic,1,1.7,0.7,1.0,1.7,0.7,1.0,4.6,2.4,2.2,13,1,4,0,2,0,2,1,11,0,3,1,1,0,2,1,2,1,1,0,-1,-7,1,-4,2,-1,5,-1,-11,-6,-5,-3,5,9,3,3,7,7,3,6,14,9,8,5,-5,-4,-4,-8,-4,-2,-6,-1,0,-10,0,-9,1,0,1,0,1,-1,6,0,6
375,Sky Bet Championship,"3:00pm, Saturday 3rd August 2019.",NaN,Brentford,Birmingham City,1,2.1,0.2,1.9,2.1,0.2,1.9,2.7,1.2,1.5,15,0,4,0,1,0,1,1,14,0,4,0,1,0,2,1,1,0,0,-1,-2,-14,-1,-6,1,1,9,-4,-2,-8,-1,-3,10,17,7,7,14,10,0,9,7,9,4,5,-3,0,-1,0,-3,-3,-10,0,-6,-10,-5,-9,1,-1,2,0,-1,1,6,-10,16
376,Sky Bet Championship,"3:00pm, Saturday 3rd August 2019.",NaN,Millwall,Preston North End,1,1.8,1.2,0.6,1.8,1.2,0.6,3.7,2.6,1.1,13,1,6,1,2,1,2,0,15,1,5,1,1,0,1,0,-2,1,1,1,1,-1,2,-2,4,2,2,1,-2,-1,-1,-3,1,4,1,1,15,8,11,4,4,3,2,4,-8,-6,-9,-4,-6,-6,-3,-5,-8,-7,-3,0,2,2,0,0,1,-1,4,-1,5
377,Sky Bet Championship,"3:00pm, Saturday 3rd August 2019.",NaN,Reading,Sheffield Wednesday,1,0.3,1.2,-0.9,0.3,1.2,-0.9,5.8,3.2,2.6,12,1,3,1,1,0,3,1,15,1,5,0,2,0,2,0,-3,-2,-1,1,-3,-3,-5,-1,4,2,2,4,-3,-2,-2,-1,7,5,8,8,1,9,3,6,7,4,4,2,-4,-4,0,-4,-6,-4,-4,-5,-5,-5,-3,-6,-1,2,-3,0,0,0,-3,1,-4
378,Sky Bet Championship,"3:00pm, Saturday 3rd August 2019.",NaN,Stoke City,Queens Park Rangers,1,2.5,1.0,1.5,2.5,1.0,1.5,5.5,3.3,2.2,9,1,3,1,2,1,1,0,11,0,5,0,2,0,2,0,-2,-2,0,-1,2,-1,3,-3,4,3,3,2,-1,-5,3,-3,0,5,0,0,0,2,0,2,4,7,0,5,-11,-6,-8,-4,-7,-6,-4,-5,-7,-13,-8,-5,0,2,-2,0,0,0,-10,6,-16
379,Sky Bet Championship,"3:00pm, Saturday 3rd August 2019.",NaN,Swansea City,Hull City,1,1.6,0.1,1.5,1.6,0.1,1.5,4.6,0.2,4.4,13,1,5,1,2,1,1,0,10,0,3,0,0,0,3,0,3,2,2,-2,2,-6,1,-10,3,-4,1,-2,-1,-20,1,-10,2,9,1,1,4,10,2,7,4,22,1,11,-8,-7,-5,0,-5,0,-3,0,-7,0,-5,0,3,-2,5,0,0,0,7,-7,14
380,Sky Bet Championship,"3:00pm, Saturday 3rd August 2019.",NaN,Wigan Athletic,Cardiff City,1,2.4,2.2,0.2,2.4,2.2,0.2,3.2,4.2,-1.0,13,0,6,1,3,0,1,0,13,1,5,1,2,1,1,0,0,1,1,0,-2,3,-1,1,3,3,5,1,-2,1,-1,-2,5,0,5,5,6,5,0,8,9,1,7,4,-11,-7,-6,-9,-6,-5,-7,-3,-8,-8,-9,-4,2,1,1,1,0,1,2,-2,4
381,Sky Bet Championship,"5:30pm, Saturday 3rd August 2019.",NaN,Nottingham Forest,West Bromwich Albion,1,1.0,1.0,0.0,1.0,1.0,0.0,4.1,2.1,2.0,11,1,3,1,1,0,2,1,15,1,5,1,1,1,1,1,-4,-2,0,1,-6,-4,-3,-2,1,2,2,3,-6,-2,-3,-2,9,10,6,6,9,12,4,9,9,4,5,6,0,-3,0,-1,-2,-6,-5,-5,0,-8,0,

In [33]:
bund_Test=Covid_Test(bund_Test)

0
0
0
0
0
0
0
1
1
5
0
0
0
0
0
0
0
0
0
0
1
12
1
0
0
0
0
0
0
0
0
0
1
2
0
0
0
0
0
1
0
0
0
0
0
2
1
0
0
0
0
0
0
0
0
0
2
4
1
0
0
0
0
0
0
0
0
0
1
2
0
0
0
0
0
1
0
0
0
0
0
2
1
0
0
0
0
0
0
0
0
0
0
6
1
0
0
0
0
0
0
0
0
1
1
3
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
1
2
0
0
0
0
0
0
0
0
0
1
0
9
0
0
0
0
0
0
0
0
0
0
1
5
1
0
0
0
0
0
0
0
0
1
1
1
0
1
0
3
3
0
0
0
0
0
0
0
3
0
1
0
0
0
0
0
0
0
0
0
6
1
0
0
0
0
0
0
0
0
0
1
2
0
0
0
0
0
1
0
0
0
0
0
2
1
0
0
0
0
0
0
0
0
0
0
6
1
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
1
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
2
4
1
0
0
0
0
0
0
0
0
0
1
104
0
0
0
0
0
0
0
0
0
0
1
5
1
0
0
0
0
0
0
0
0
1
0
2
0
0
0
0
0
0
1
0
0
0
1
1
1
0
0
0
0
0
0
0
1
0
0
2
0
0
0
0
1
0
0
0
0
0
1
1
0
1
0
0
0
0
0
0
0
1
0
2
0
0
0
0
0
0
1
0
0
0
1
1
1
0
0
0
0
0
0
0
0
1
0
3
0
0
0
0
0
0
0
0
0
0
0
104
League                  Sky Bet Championship
Date                     2020-03-08 00:00:00
Time                                    1500
HT                              Derby County
AT                          Blackburn Rovers
Colx

In [13]:
bund_Test

,League,Date,Time,HT,AT,Colx,HT_HPoints_5,AT_APoints_5,HA_HAPointsDiff,HT_TotalPoints_5,AT_TotalPoints_5,HA_TotalPointsDiff,SQuality_H5,SQuality_A5,SQuality_Diff5,HS_5,Last_H_S,HST_5,Last_H_ST,HScrd_5,Last_H_Scrd,HConcd_5,Last_H_Concd,AS_5,Last_A_S,AST_5,Last_A_ST,AScrd_5,Last_A_Scrd,AConcd_5,Last_A_Concd,S_Diff_H-A_5,ST_Diff_H-A_5,Scrd_Diff_H-A_5,Concd_Diff_H-A_5,TacklesH,TacklesA,Tackles_Diff,WinCounterTotal_H,WinCounterTotal_A,ScrdCounterTotal_H,ScrdCounterTotal_A,ConcdCounterTotal_H,ConcdCounterTotal_A,WinPercTotal_H,WinPercTotal_A,ScrdPercTotal_H,ScrdPercTotal_A,ConcdPercTotal_H,ConcdPercTotal_A,WinCounterH_H,WinCounterA_A,ScrdCounterH_H,ScrdCounterTotalA_A,ConcdCounterH_H,ConcdCounterA_A,WinPercH_H,WinPercA_A,ScrdPercH_H,ScrdPercTotalA_A,ConcdPercH_H,ConcdPercA_A,STForm_H5,STForm_A5,STForm_Diff5,CCCForm_H5,CCCForm_A5,CCCForm_Diff5,PassForm_H5,PassForm_A5,PassForm_Diff5,GoalForm_H5,GoalForm_A5,GoalForm_Diff5,PointsForm_H5,PointsForm_A5,PointsForm_Diff5,Won/NotWon
0,Dutch Eredivisie,2019-02-10,1330,Emmen,ADO Den Haag,1,0.6,1.5,-0.9,0.6,1.5,-0.9,4.2,4.0,0.2,8,0,2,0,1,0,3,1,13,1,5,1,2,0,1,0,-5,-3,-1,2,66,66,0,-1,1,-1,5,-5,1,-5,5,-5,25,-40,5,-2,-1,1,-1,0,-2,0,-100,0,400,0,0,-2,1,-3,-1,1,-2,4,-5,9,0,0,0,0,0,0,1
1,Dutch Eredivisie,2019-02-10,1545,FC Utrecht,PSV Eindhoven,1,1.4,1.8,-0.4,1.4,1.8,-0.4,4.0,4.3,-0.3,12,0,4,0,1,1,3,1,16,1,8,1,3,1,1,0,-4,-4,-2,2,61,61,0,-4,2,1,5,-5,1,-20,10,5,100,-35,5,-2,-2,-1,1,0,-1,0,-100,0,900,0,100,0,2,-2,0,2,-2,2,8,-6,0,2,-2,-1,1,-2,0
2,Dutch Eredivisie,2019-02-15,1900,ADO Den Haag,PEC Zwolle,1,1.2,1.5,-0.3,1.2,1.5,-0.3,4.5,4.1,0.4,13,0,5,0,2,0,2,1,14,0,5,1,2,0,1,0,-1,0,0,1,62,62,0,-1,-1,5,4,-1,-2,-5,-5,29,19,-5,-10,1,-2,2,2,2,-1,0,0,0,0,0,0,0,0,0,0,0,0,-4,5,-9,0,0,0,-1,0,-1,1
3,Dutch Eredivisie,2019-02-16,1730,AZ Alkmaar,VVV-Venlo,1,2.5,0.8,1.7,2.5,0.8,1.7,4.8,2.7,2.1,17,0,7,1,3,0,0,0,13,1,5,1,2,0,3,0,4,2,1,-3,59,59,0,4,1,5,1,4,-4,19,5,33,5,19,-19,2,2,2,2,2,2,100,0,100,0,133,0,2,0,2,1,0,1,8,-2,10,1,0,1,2,0,2,1
4,Dutch Eredivisie,2019-02-16,1845,Heracles Almelo,Fortuna Sittard,1,1.3,0.6,0.7,1.3,0.6,0.7,2.0,3.2,-1.2,12,0,3,0,1,0,1,0,11,1,4,1,2,1,2,0,1,-1,-1,-1,62,62,0,1,1,1,1,1,-4,5,5,5,5,5,-19,1,-2,1,-2,1,0,0,0,0,0,0,0,-2,-1,-1,-1,0,-1,-4,-1,-3,-2,0,-2,0,1,-1,1
5,Dutch Eredivisie,2019-02-16,1845,Heerenveen,PSV Eindhoven,1,0.8,1.5,-0.7,0.8,1.5,-0.7,2.6,4.4,-1.8,10,0,5,0,2,0,2,0,18,1,7,0,3,0,1,1,-8,-2,-1,1,61,61,0,-5,-1,2,5,-5,-1,-29,-5,10,100,-62,-5,-2,-2,1,2,-1,-2,0,-100,0,500,0,-50,1,1,0,1,1,0,-1,7,-8,0,1,-1,0,0,0,0
6,Dutch Eredivisie,2019-02-16,1945,Vitesse Arnhem,Willem II,1,2.1,2.1,0.0,2.1,2.1,0.0,5.9,3.6,2.3,12,0,4,0,2,0,2,0,10,0,4,1,1,0,1,1,2,0,1,1,60,60,0,-3,-2,5,5,-5,-2,-14,-10,48,24,-62,-10,-1,-2,2,2,0,-1,0,0,0,0,0,0,-3,-1,-2,0,-1,1,-4,1,-5,-1,-1,0,-1,0,-1,1
7,Dutch Eredivisie,2019-02-17,1115,Ajax,NAC Breda,1,2.5,0.2,2.3,2.5,0.2,2.3,3.0,0.8,2.2,21,1,8,0,3,0,2,0,13,0,3,0,1,0,2,1,8,5,2,0,58,58,0,-1,-4,-1,-2,-1,-5,-5,-19,-5,-10,-5,-24,1,-2,2,-1,2,-1,0,0,0,0,0,0,3,-3,6,1,0,1,8,-6,14,1,-2,3,-1,-1,0,1
8,Dutch Eredivisie,2019-02-17,1330,Excelsior,Emmen,1,1.8,1.6,0.2,1.8,1.6,0.2,4.9,4.2,0.7,12,1,4,1,2,0,2,1,9,1,3,1,2,1,3,0,3,1,0,-1,50,50,0,-1,1,4,1,-2,-5,-5,5,19,5,-10,-43,2,-2,2,2,0,-1,0,0,0,0,0,0,-2,-1,-1,1,1,0,-2,4,-6,-1,1,-2,0,1,-1,1
9,Dutch Eredivisie,2019-02-17,1330,Groningen,Feyenoord,1,2.4,0.9,1.5,2.4,0.9,1.5,5.8,3.2,2.6,11,0,4,0,2,0,1,0,19,1,9,1,3,1,2,0,-8,-5,-1,-1,51,51,0,2,1,4,5,-3,1,10,5,19,24,-14,5,2,1,2,2,-1,-1,0,-300,0,500,0,-100,0,5,-5,0,2,-2,-10,12,-22,1,1,0,1,0,1,1


In [34]:
len(bund_Test)

323

In [79]:
#bund_Test.iloc[np.where(bund_Test['HT']=='Sampdoria')[0]]
#b_TT = bund_Test[:88]
#bund_Test.iloc[np.where(bund_Test['HT']=='Sassuolo')[0]]
#b_Test2 = bund_Test[239:]
bund_Test=bund_Test[85:266]

In [80]:
bund_Test= bund_Test.reset_index(drop=True)
#bund_Train = pd.concat([bund_Train,b_TT],sort=False).sort_values(['Date','Time'],ascending=[True,True]).reset_index(drop=True)


In [82]:
bund_Test.head(2)

,League,Date,Time,HT,AT,H_xPts_Total,H_xPts_H,A_xPts_Total,A_xPts_A,xPoints_Diff_Total,xPoints_Diff_H-A,HT_HPoints_5,AT_APoints_5,HA_HAPointsDiff,HxG_5,HxPTS_5,HPass%_5,HPPDA_5,HDEEP_5,HCCC_5,HxConcd_5,AxG_5,AxPTS_5,APass%_5,APPDA_5,ADEEP_5,ACCC_5,AxConcd_5,xG_Diff_5,xPTS_Diff_5,Pass%_Diff_5,PPDA_Diff_5,DEEP_Diff_5,CCC_Diff_5,xConcd_Diff_5,SQuality_H5,SQuality_A5,SQuality_Diff5,HS_5,HST_5,HScrd_5,HConcd_5,AS_5,AST_5,AScrd_5,AConcd_5,S_Diff_H-A_5,ST_Diff_H-A_5,Scrd_Diff_H-A_5,Concd_Diff_H-A_5,WinCounterTotal_H,WinCounterTotal_A,WinCounter_H,WinCounter_A,ScrdCounterTotal_H,ScrdCounterTotal_A,ScrdCounter_H,ScrdCounter_A,ConcdCounterTotal_H,ConcdCounterTotal_A,ConcdCounter_H,ConcdCounter_A,Lucky_H5,Lucky_A5,Lucky_Diff5,Unlucky_H5,Unlucky_A5,Unlucky_Diff5,STForm_H5,STForm_A5,STForm_Diff5,CCCForm_H5,CCCForm_A5,CCCForm_Diff5,DEEPForm_H5,DEEPForm_A5,DEEPForm_Diff5,PPDAForm_H5,PPDAForm_A5,PPDAForm_Diff5,PassForm_H5,PassForm_A5,PassForm_Diff5,DeForm_H5,DeForm_A5,DeForm_Diff5,AtkForm_H5,AtkForm_A5,AtkForm_Diff5,Form1_H5,Form1_A5,Form1_Diff5,Form2_H5,Form2_A5,Form2_Diff5,Form3_H5,Form3_A5,Form3_Diff5,NewForm_H5,NewForm_A5,NewForm_Diff5,Won/NotWon,ODDH_Aver.,ODDD_Aver.,ODDA_Aver.
0,Serie_A,2019-10-20,1400,Sampdoria,Roma,1.1,1.3,1.8,1.6,-0.7,-0.3,1.0,2.3,-1.3,1.1,1.2,75,13,4,0,1.3,2.1,1.9,85,8,13,3,1.2,-1.0,-0.7,-10,5,-9,-3,0.1,0.8,1.3,-0.5,14,3,1,2,18,5,1,1,-4,-2,0,1,-3,-1,-1,2,-1,2,2,3,-3,-1,-1,1,-3.8,-0.5,-3.3,0.3,1.6,-1.3,-1.6,0.0,-1.6,-1.4,0.7,-2.1,-2,7,-9,0,-3,3,-6,6,-12,28,42,-14,-26,54,-80,1.0,3.2,-2.2,1.6,1.4,0.2,1.0,2.8,-1.8,6.4,9.4,-3.0,0,3.28,3.54,2.17
1,Serie_A,2019-10-20,1700,Parma,Genoa,1.2,1.7,1.0,1.0,0.2,0.7,1.5,0.3,1.2,1.4,1.4,77,11,4,1,1.3,1.2,1.1,82,13,6,1,1.5,0.2,0.3,-5,-2,-2,0,-0.2,1.0,0.9,0.1,13,3,1,1,12,4,1,2,1,-1,0,-1,-1,-5,2,-3,-1,1,3,-1,-2,-2,-1,-3,-0.5,-4.5,4.0,1.5,4.8,-3.3,-2.6,0.6,-3.2,-0.7,0.1,-0.8,-4,2,-6,-1,2,-3,-4,3,-7,-24,24,-48,-27,-19,-8,1.7,1.2,0.5,1.3,1.7,-0.4,2.2,0.9,1.3,5.5,6.0,-0.5,1,2.51,3.21,2.95


In [37]:
ok = 'Championship_Odds'
portal = pd.read_excel('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//Portal_Odds//'+ok+'.xlsx')

In [38]:
bund_Test = Portal_addodds(portal,bund_Test,'Championship')

['Barnsley', 'Birmingham City', 'Blackburn Rovers', 'Brentford', 'Bristol City', 'Cardiff City', 'Charlton Athletic', 'Derby County', 'Fulham', 'Huddersfield Town', 'Hull City', 'Leeds United', 'Luton Town', 'Middlesbrough', 'Millwall', 'Nottingham Forest', 'Preston North End', 'Queens Park Rangers', 'Reading', 'Sheffield Wednesday', 'Stoke City', 'Swansea City', 'West Bromwich Albion', 'Wigan Athletic']
Barnsley
Birmingham City
Blackburn Rovers
Brentford
Bristol City
Cardiff City
Charlton Athletic
Derby County
Fulham
Huddersfield Town
Hull City
Leeds United
Luton Town
Middlesbrough
Millwall
Nottingham Forest
Preston North End
Queens Park Rangers
Reading
Sheffield Wednesday
Stoke City
Swansea City
West Bromwich Albion
Wigan Athletic


In [39]:
bund_Test[['HT','AT','Date','ODDH_Aver.','ODDD_Aver.','ODDA_Aver.']].tail(20)

,HT,AT,Date,ODDH_Aver.,ODDD_Aver.,ODDA_Aver.
303,Fulham,Sheffield Wednesday,2020-07-18,2,3.41,3.92
304,Hull City,Luton Town,2020-07-18,2.57,3.49,2.68
305,Middlesbrough,Cardiff City,2020-07-18,3.23,3.17,2.36
306,Preston North End,Birmingham City,2020-07-18,2.07,3.36,3.72
307,Queens Park Rangers,Millwall,2020-07-18,4.81,3.68,1.76
308,Swansea City,Bristol City,2020-07-18,1.88,3.6,4.19
309,Derby County,Leeds United,2020-07-19,2.85,3.47,2.45
310,Barnsley,Nottingham Forest,2020-07-19,2.62,3.27,2.79
311,Birmingham City,Derby County,2020-07-22,2.22,3.46,3.25
312,Brentford,Barnsley,2020-07-22,1.4,5.15,7.11


In [30]:
bund_Train.to_excel('SkyScott_Train.xlsx',index=False)
bund_Test.to_excel('SkyScott_Test.xlsx',index=False)

In [6]:
oooo = open('PortalTeams_Eredivisie.pickle','rb')
okok = pickle.load(oooo)
oooo.close()